# Viewing Vibrations from Hessian Calculation

## Name of the output file

In [1]:
input_file = "hessian.log"

## Import python packages

In [16]:
import numpy as np
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# let plotly offine
plotly.offline.init_notebook_mode(connected=True)
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import py3Dmol
import glob, os, itertools, re

## Define functions

### Load user-defined function to pull out the vibration mode information

In [3]:
def vib_mode_info(input_file):
    # Define variables used throughout the function
    nu_atom = 1
    nu_vib = 1
    list_dict_vib = []

    # Define temporary file nams
    vib_coord_file = "vib_coord.txt"
    vib_coord_file_tmp = "vib_coord.txt.tmp"
    vib_list_file = "vib_list.txt"
    nu_atom_file = "nu_atom.txt"
    nu_vib_file = "nu_vib.txt"

    command_1 = "grep 'Number of atoms:' " + input_file + " > " + nu_atom_file
    command_2 = "grep 'VIBRATIONAL MODES ARE USED IN THERMOCHEMISTRY' " + input_file + " > " + nu_vib_file
    command_3 = "sed -n '/MODE FREQ(CM\*\*-1)  SYMMETRY  RED. MASS  IR INTENS./,/THERMOCHEMISTRY/p' " + input_file + " > " + vib_list_file
    command_4 = "sed -n '/ANALYZING SYMMETRY OF NORMAL MODES/,/REFERENCE ON SAYVETZ CONDITIONS/p' " + input_file + " > " + vib_coord_file_tmp
    command_5 = "sed -n '/1           2           3           4           5/,/REFERENCE ON SAYVETZ CONDITIONS/p' " + vib_coord_file_tmp + " > " + vib_coord_file
    
    os.system(command_1)
    os.system(command_2)
    os.system(command_3)
    os.system(command_4)
    os.system(command_5)

    with open(nu_atom_file, "r") as f:
        list_txt_nu_atom = f.read().split()
        nu_atom = int(list_txt_nu_atom[-1])

    with open(nu_vib_file, "r") as f:
        list_txt_nu_vib_1 = f.read().split("-")
        list_txt_nu_vib_2 = list_txt_nu_vib_1[-1].split()
        nu_vib = int(list_txt_nu_vib_2[0])

    with open(vib_list_file, "r") as f:
        for line in itertools.islice(f, 1, nu_vib + 1):
            
            list_line = line.split()
            
            dict_vib = {}
            dict_vib["mode"] = int(list_line[0])
            dict_vib["frequency"] = float(list_line[1])
            dict_vib["symmetry"] = list_line[2]
            dict_vib["red_mass"] = float(list_line[3])
            dict_vib["ir_intens"] = float(list_line[4])

            list_dict_vib.append(dict_vib)

    # All information for a vibration is saved in the variable list_dict_vib.

    nu_coord_line_per_geom = nu_atom * 3

    with open(vib_coord_file, "r") as f:
        list_line_1 = f.read().splitlines()
        list_line_2 = list(filter(None, list_line_1))     # complete list without empty lines

        vib_coord_len = len(list_line_2)
        nu_line_per_block = nu_coord_line_per_geom + 5 + 8    
        nu_block = int(vib_coord_len/nu_line_per_block)   

        list_dict_geom = []

        for i in range(0, nu_block):   # loop over 23 blocks
            ini = i * 127
            end = (i + 1) * 127
            list_line_block = list_line_2[ini:end]    # a blok of frequencies and coordinates
            list_vib_mode = list_line_block[0].split()
            nu_freq = len(list_vib_mode)    # number of vibration modes per block, corresponding to number of geometries
            list_coord_block = list_line_block[5:5 + nu_coord_line_per_geom]

            list_line_comp_block =[]

            for line in list_coord_block:
                line_comp = line.split()
                list_line_comp_block.append(line_comp)
            
            # loop over the frequency number in a block
            for k in range(0, nu_freq):
                dict_mode_geom = {}
                dict_mode_geom["mode"] = int(list_vib_mode[k])

                freq_ind = k - nu_freq

                list_geom = []    # list of atom coordinates of one structure

                # loop over each atom in a column (each structure/column)
                for j in range(0, nu_atom):
                    ind = 3 * j
                    x_ind = ind + 0
                    y_ind = ind + 1
                    z_ind = ind + 2

                    dict_atom_coord = {}
                    dict_atom_coord["atom"] = list_line_comp_block[x_ind][1]
                    dict_atom_coord["dx"] = float(list_line_comp_block[x_ind][freq_ind])
                    dict_atom_coord["dy"] = float(list_line_comp_block[y_ind][freq_ind])
                    dict_atom_coord["dz"] = float(list_line_comp_block[z_ind][freq_ind])
                    list_geom.append(dict_atom_coord)
                
                dict_mode_geom["geometry"] = list_geom
            
                list_dict_geom.append(dict_mode_geom)
                
    # At this step, we've gotten the geometry for each frequency mode. 
    # All is save in the variable list_dict_geom.
    # Also, the vibration mode information is saved in the varible list_dict_vib.

    command_6 = "rm -f vib_coord.txt.tmp vib_coord.txt vib_list.txt nu_atom.txt nu_vib.txt"
    os.system(command_6)

    # Merge two lists
    for i in range(len(list_dict_vib)):
        list_dict_vib[i]["geometry"] = list_dict_geom[i].get("geometry")

    return(list_dict_vib)

In [4]:
def vibrate(mode):
    xyzview = py3Dmol.view(width=500,height=500)
    xyzview.addModel(xyz_vib_list[mode - 1],'xyz',{'vibrate': {'frames':10,'amplitude':5}})
    xyzview.setStyle({'stick':{}})
    xyzview.setBackgroundColor('white')
    xyzview.animate({'loop': 'BackandForth'})
    xyzview.zoomTo()
    xyzview.show()

## Read the initial coordinates into a list

In [ ]:
# Third method
# Read initial coordinate from a Gamess input file

input_name = input_file[:-4] + ".inp"
init_coord_file = glob.glob(input_name)

atom_pattern = "^\s+[A-Z][a-z]?"
nu_pattern = "\s+-?[0-9]{1,3}\.[0-9]+"
coord_srch_pattern = atom_pattern + 4*nu_pattern + "\s*$"

list_dict_orig_coord = []

with open(init_coord_file, "r") as f:
    list_coord = f.read().splitlines()
    
    for i in list_coord:
        if re.search(coord_srch_pattern, i):
            k = i.split()
            dict_atom_coord = {}    
            dict_atom_coord["atom"] = k[0]
            dict_atom_coord["x"] = float(k[1])
            dict_atom_coord["y"] = float(k[2])
            dict_atom_coord["z"] = float(k[3])
            
            list_dict_orig_coord.append(dict_atom_coord)

## Read the log file

In [6]:
list_dict_vib = vib_mode_info(input_file)

### Create new list for each vibration mode with x, y, z, dx, dy, dz

In [7]:
# Loop through each mode/each geometry in the hessian calculation
# Add dx, dy, dz to the "geometry" in the dictionary of vibration information

for i in list_dict_vib:
    each_vib_geom = i.get("geometry")
    
    for k in range(len(each_vib_geom)):
        each_vib_geom[k]["x"] = list_dict_orig_coord[k].get("x")
        each_vib_geom[k]["y"] = list_dict_orig_coord[k].get("y")
        each_vib_geom[k]["z"] = list_dict_orig_coord[k].get("z")

In [8]:
list_xyz_name = []

for i in list_dict_vib:
    each_vib_geom = i.get("geometry")
    
    xyz_file = "hess_vib_mode_" + str(i.get("mode")) + ".xyz"
    list_xyz_name.append(xyz_file)
    
    with open(xyz_file, "w") as f:
        f.write('{}\n\n'.format(len(each_vib_geom)))
        
        for k in each_vib_geom:
            f.write('{:<4s}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}\n'.format(k.get("atom"), k.get("x"), k.get("y"), k.get("z"), k.get("dx"), k.get("dy"), k.get("dz")))
            

In [9]:
xyz_vib_list = []

for i in list_dict_vib:
    each_vib_geom = i.get("geometry")
    s = str()
    s = '{}\n\n'.format(len(each_vib_geom))
    
    for k in each_vib_geom:
        s = s + '{:<4s}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}{:>15.6f}\n'\
        .format(k.get("atom"), k.get("x"), k.get("y"), k.get("z"), k.get("dx"), k.get("dy"), k.get("dz"))
    
    xyz_vib_list.append(s)

## Now ... the plot: Intensity vs. Wave number

In [14]:
x = []
y = []
mode = []

for i in range(0, len(list_dict_vib)):
    x.append(list_dict_vib[i].get("frequency"))
    y.append(list_dict_vib[i].get("ir_intens"))
    mode.append("mode: " + str(i + 1))

In [15]:
trace = go.Scatter(
    x = np.array(x),
    y = np.array(y),
    mode = "lines+markers",
    line = dict(color = "red"),
    text = mode
)

data = [trace]
iplot(data)

## This is how the atoms dance ~-~ ^.^ ~-~

In [12]:
interact(vibrate, mode = widgets.IntSlider(min=1,max=114,step=1,value=1))

interactive(children=(IntSlider(value=1, description='mode', max=114, min=1), Output()), _dom_classes=('widget…

<function __main__.vibrate(mode)>